In this Jupyter Notebook, I compare the convergence of eigenvector continuation (Amp-ccevc) amplitudes as a start guess in CC calculations with the convergence of CC amplitudes using either 
- the previous geometry with procrustes orbitals with respect to the previous geometry
- the previous geometry with canonical orbitals 
- the previous geometry with procrustes orbitals with respect to a reference geometry
- MP2 amplitudes
- different number of EVC ampltidues (machine learned, 50%, number of sample geometries)

In [ ]:
"""
Get approximate AMP-CCEVC amplitudes for HF molecule in small basis
"""
import sys
sys.path.append("../libraries")
from rccsd_gs import *

from func_lib import *
from numba import jit
from matrix_operations import *
from helper_functions import *
basis = '6-31G'
charge = 0
molecule=lambda x:  "H 0 0 0; F 0 0 %f"%x
refx=[1.75]
print(molecule(*refx))
reference_determinant=get_reference_determinant(molecule,refx,basis,charge)
sample_geometry=[[np.linspace(1.5,5,6),np.linspace(1.5,2,6)],[np.linspace(1.5,2,16),np.linspace(4.5,5,6)]]
sample_geom1=np.linspace(1.5,5,6)
import pickle
geom_alphas1=np.linspace(1.2,5,77)
geom_alphas=[[x] for x in geom_alphas1]
energy_dict={}
energy_dict["xval"]=geom_alphas1
energy_dict["CCSD"]=CCSD_energy_curve(molecule,geom_alphas,basis)
energies_WF=[[],[]]
energies_AMP=[[],[]]
energies_AMPred=[[],[]]

sample_geom=[[x] for x in sample_geom1]
sample_geom1=np.array(sample_geom).flatten()
t1s,t2s,l1s,l2s,sample_energies=setUpsamples(sample_geom,molecule,basis,reference_determinant,mix_states=False,type="procrustes")
evcsolver=EVCSolver(geom_alphas,molecule,basis,reference_determinant,t1s,t2s,l1s,l2s,sample_x=sample_geom,mix_states=False)
E_AMP_full=evcsolver.solve_AMP_CCSD(occs=1,virts=1)
E_AMP_red=evcsolver.solve_AMP_CCSD(occs=1,virts=0.5)
energies_AMP[i].append(E_AMP_full)
energies_AMPred[i].append(E_AMP_red)
t1s_reduced=E_AMP_red.t1s_final
t2s_reduced=E_AMP_red.t2s_final
t1s_full=E_AMP_red.t1s_final
t2s_full=E_AMP_full.t2s_final